### Knock41: 基本的な Directory を生成しよう
> #### Master data と input data
> 明確な基準はないが、更新頻度によって仕訳するとよい。
> - 更新頻度-高: input data
> - 更新頻度-低: master data

In [1]:
# Directory path の定義
import os
data_dir = 'data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '10_output')
master_dir = os.path.join(data_dir, '99_master_dir')
print(input_dir)

data\0_input


`os.mkdir()` と `os.makedirs()` の違い
- `os.makedirs()` は、再帰的に中間の Directory を自動で生成してくれる。

In [2]:
# Directory の生成
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)